# Testing Tensorflow on macOS and with Apple Silicon

**Note:** If running this on Mac, execute it in a `tensorflow_macos` environment.

To install `tensorflow_macos`, follow the installation steps in the repository: https://github.com/apple/tensorflow_macos

In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

**Note:** The following three cells won't work on Colab, they require the `tensorflow_macos` fork of Tensorflow.

In [3]:
from tensorflow.python.compiler.mlcompute import mlcompute

In [5]:
mlcompute.is_apple_mlc_enabled()

True

In [6]:
mlcompute.is_tf_compiled_with_apple_mlc()

True

# Test 1 - Basic CNN

First we'll need some data (comment/uncomment the cell below if data is required/not required).

In [10]:
# Download data (if needed)
# Data is a subset of Food101 - https://www.kaggle.com/dansbecker/food-101
!wget -q https://www.dropbox.com/s/hqjg1j8teqimvyx/10_food_classes_10_percent.zip
!wget -q https://www.dropbox.com/s/7umiyjiimhqyxj0/10_food_classes_all_data.zip

import zipfile

def unzip_data(filename):
  """
  Unzips filename into the current working directory.

  Args:
    filename (str): a filepath to a target zip folder to be unzipped.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

unzip_data("10_food_classes_all_data.zip")
unzip_data("10_food_classes_10_percent.zip")

In [11]:
# Smaller dataset
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

# # Larger dataset
# train_dir = "10_food_classes_all_data/train"
# test_dir = "10_food_classes_all_data/test"

In [12]:
import os
os.listdir(train_dir)

['ice_cream',
 'chicken_curry',
 'steak',
 'sushi',
 'chicken_wings',
 'grilled_salmon',
 'hamburger',
 'pizza',
 'ramen',
 'fried_rice']

In [13]:
# Create data inputs (need batch size of ~4 for transfer learning on M1)
IMG_SIZE = (224, 224)
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=IMG_SIZE,
                                                                 label_mode="categorical",
                                                                 batch_size=32)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                image_size=IMG_SIZE,
                                                                label_mode="categorical",
                                                                batch_size=32)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [14]:
# Make a simple conv net
model = tf.keras.Sequential([
    # Normalize pixel inputs
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    
    # Create a basic conv net (same as https://poloclub.github.io/cnn-explainer/)
    tf.keras.layers.Conv2D(10, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.Conv2D(10, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(10, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.Conv2D(10, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax")
])

# Compile
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit
model.fit(train_data,
          epochs=5,
          validation_data=test_data)

Epoch 1/5
24/24 [==============================] - 13s 538ms/step - loss: 2.2999 - accuracy: 0.0986 - val_loss: 2.2509 - val_accuracy: 0.1364
Epoch 2/5
24/24 [==============================] - 12s 512ms/step - loss: 2.2061 - accuracy: 0.1818 - val_loss: 2.2404 - val_accuracy: 0.1908
Epoch 3/5
24/24 [==============================] - 12s 499ms/step - loss: 1.9742 - accuracy: 0.3134 - val_loss: 2.3379 - val_accuracy: 0.1656
Epoch 4/5
24/24 [==============================] - 12s 498ms/step - loss: 1.6772 - accuracy: 0.4392 - val_loss: 2.5653 - val_accuracy: 0.1848
Epoch 5/5
24/24 [==============================] - 13s 517ms/step - loss: 1.1239 - accuracy: 0.6585 - val_loss: 3.0334 - val_accuracy: 0.1684


# Test 2 - Transfer Learning

We need a smaller batch sizer for transfer learning on M1 chips (takes up less memory).

In [15]:
# Create data inputs (need batch size of ~4 for transfer learning on M1)
IMG_SIZE = (224, 224)
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=IMG_SIZE,
                                                                 label_mode="categorical",
                                                                 batch_size=4)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                image_size=IMG_SIZE,
                                                                label_mode="categorical",
                                                                batch_size=4)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [16]:
# Try a transfer learning model - https://arxiv.org/abs/1905.11946
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# Freeze the base model
base_model.trainable = False

# Create inputs and outputs
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

# Compile
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit
model.fit(train_data,
          epochs=20,
          batch_size=4, # need smaller batch size for M1
          steps_per_epoch=len(train_data),
          validation_data=test_data,
          validation_steps=int(0.25 * len(test_data)))

16711680/16705208 [==============================] - 4s 0us/step
Epoch 1/20
188/188 [==============================] - 23s 112ms/step - loss: 2.2342 - accuracy: 0.1864 - val_loss: 2.0523 - val_accuracy: 0.3413
Epoch 2/20
188/188 [==============================] - 20s 108ms/step - loss: 1.6243 - accuracy: 0.5509 - val_loss: 4.3734 - val_accuracy: 0.2500
Epoch 3/20
188/188 [==============================] - 20s 108ms/step - loss: 1.3472 - accuracy: 0.6191 - val_loss: 1.8725 - val_accuracy: 0.3686
Epoch 4/20
188/188 [==============================] - 20s 106ms/step - loss: 1.1847 - accuracy: 0.6385 - val_loss: 2.0803 - val_accuracy: 0.3125
Epoch 5/20
188/188 [==============================] - 20s 107ms/step - loss: 1.0733 - accuracy: 0.7148 - val_loss: 1.7970 - val_accuracy: 0.4343
Epoch 6/20
188/188 [==============================] - 20s 107ms/step - loss: 1.0058 - accuracy: 0.7250 - val_loss: 1.9966 - val_accuracy: 0.2949
Epoch 7/20
188/188 [==============================] - 20s 107ms/s

# Test 3 - GitHub Benchmark

Benchmark from GitHub: https://github.com/apple/tensorflow_macos/issues/25

In [17]:
# The benchmark requires tensorflow_datasets (you may need to restart the kernel after this)
!pip install tensorflow_datasets

You should consider upgrading via the '/Users/loyio/miniforge3/envs/tf24/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

tf.enable_v2_behavior()

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

from tensorflow.python.compiler.mlcompute import mlcompute
mlcompute.set_mlc_device(device_name='gpu')


(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

batch_size = 128

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

Train on 469 steps, validate on 79 steps
Epoch 1/12
469/469 [==============================] - ETA: 0s - batch: 234.0000 - size: 1.0000 - loss: 0.6969 - accuracy: 0.8657

/Users/loyio/miniforge3/envs/tf24/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


469/469 [==============================] - 21s 43ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.6969 - accuracy: 0.8657 - val_loss: 0.1704 - val_accuracy: 0.9644
Epoch 2/12
469/469 [==============================] - 21s 43ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.1222 - accuracy: 0.9706 - val_loss: 0.1102 - val_accuracy: 0.9778
Epoch 3/12
469/469 [==============================] - 21s 43ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0783 - accuracy: 0.9801 - val_loss: 0.0821 - val_accuracy: 0.9776
Epoch 4/12
469/469 [==============================] - 21s 43ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0657 - accuracy: 0.9834 - val_loss: 0.0675 - val_accuracy: 0.9815
Epoch 5/12
469/469 [==============================] - 21s 43ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0508 - accuracy: 0.9869 - val_loss: 0.0620 - val_accuracy: 0.9834
Epoch 6/12
469/469 [==============================] - 21s 44ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0436 - accuracy